# Data Collection: Google Places API

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Data-Collection:-Google-Places-API" data-toc-modified-id="Data-Collection:-Google-Places-API-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Data Collection: Google Places API</a></span><ul class="toc-item"><li><span><a href="#Introduction" data-toc-modified-id="Introduction-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Introduction</a></span></li><li><span><a href="#Import-libraries" data-toc-modified-id="Import-libraries-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Import libraries</a></span></li><li><span><a href="#Fetch-API-key" data-toc-modified-id="Fetch-API-key-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Fetch API key</a></span></li><li><span><a href="#Instantiate-client-object" data-toc-modified-id="Instantiate-client-object-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>Instantiate client object</a></span></li><li><span><a href="#Create-a-list-of-keywords" data-toc-modified-id="Create-a-list-of-keywords-1.5"><span class="toc-item-num">1.5&nbsp;&nbsp;</span>Create a list of keywords</a></span></li><li><span><a href="#Import-ZCTA-data" data-toc-modified-id="Import-ZCTA-data-1.6"><span class="toc-item-num">1.6&nbsp;&nbsp;</span>Import ZCTA data</a></span></li><li><span><a href="#Define-functions-for-requests" data-toc-modified-id="Define-functions-for-requests-1.7"><span class="toc-item-num">1.7&nbsp;&nbsp;</span>Define functions for requests</a></span></li><li><span><a href="#Fetch-data" data-toc-modified-id="Fetch-data-1.8"><span class="toc-item-num">1.8&nbsp;&nbsp;</span>Fetch data</a></span></li><li><span><a href="#Export-data-as-.csv-file" data-toc-modified-id="Export-data-as-.csv-file-1.9"><span class="toc-item-num">1.9&nbsp;&nbsp;</span>Export data as .csv file</a></span></li></ul></li></ul></div>

## Introduction

**Google Places API**  
To collect neighborhood commercial information, I will use the [Google Places API](https://developers.google.com/places/web-service/intro). There are several sub-categories, and I will use the [Place Search](https://developers.google.com/places/web-service/search) API. 

**Result limits**
- Results per query: 20
- Max results per location: 60
- If more than 20 results is needed, pass the value of the `next_page_token` to the pagetoken parameter of a new search to see the next set of results. 
- [Reference](https://developers.google.com/places/web-service/search)

**Python Client for Google Maps Services**  
Google Places API has a [Python Client Library](https://github.com/googlemaps/google-maps-services-python), which I will use in this project. 
- [Installation instructions](https://github.com/googlemaps/google-maps-services-python#installation)
- [Library documentation](https://developers.google.com/places/web-service/client-library)
- [Example of using library](https://github.com/googlemaps/google-maps-services-python/blob/81640b0a76fb741f228996f260a05c6e4a2cb27c/googlemaps/test/test_places.py)
- [Method source code](https://github.com/googlemaps/google-maps-services-python/blob/81640b0a76fb741f228996f260a05c6e4a2cb27c/googlemaps/places.py#L101)

## Import libraries

In [150]:
import pandas as pd
#import requests
#import datetime as dt
#import re
import time
from pathlib import Path

# Import Python Client Library for Google Maps Services
import googlemaps

## Fetch API key

- Create Google Places API key [here](https://developers.google.com/places/web-service/intro?utm_source=google&utm_medium=cpc&utm_campaign=FY18-Q2-global-demandgen-paidsearchonnetworkhouseads-cs-maps_contactsal_saf&utm_content=text-ad-none-none-DEV_c-CRE_315916118135-ADGP_Hybrid+%7C+AW+SEM+%7C+SKWS+~+Places+API-KWID_43700039136946627-kwd-475997044718-userloc_9073479&utm_term=KW_%2Bplace%20%2Bapi-ST_%2Bplace+%2Bapi&gclid=EAIaIQobChMI2Ib1w6Xo5wIVD5SzCh3eDgR9EAAYASAAEgJwXvD_BwE).
- Save the API key as `.txt` file, create a directory with the path `"~/api_keys"`, and store the API key there.

In [154]:
with open(str(Path.home() / 'api_keys/google_api.txt')) as file:
    API_KEY = file.read().replace('\n', '')

## Instantiate client object

In [9]:
# Instance of making a single call
client = googlemaps.Client(key=API_KEY, queries_per_second=2)

## Create a list of keywords

I will use the text search, input a list of key words and a list of zipcodes to fetch businesses. This key word is not exhaustive; the purpose is to sample a few common types of businesses. The decision is arbitrary.

In [136]:
LIST_OF_KEYWORDS = ['stores', 'restaurant', 'coffee shops']

## Import ZCTA data

In [117]:
zcta = pd.read_csv('../data/zcta_nyc.csv')

In [141]:
list_of_zcta = list(zcta['zcta'].values)

## Define functions for requests

In [101]:
# Define a function to send multiple requests and fetch up to 60 results
# def search_place(keyword, zipcode):
#     results = []
    
#     def _get_next_results(next_page_token=None):
#         json_result = client.places(f'{keyword} near {zipcode}', page_token=next_page_token)
#         results.extend(json_result['results'])
#         return json_result.get('next_page_token')
    
#     next_page_token = _get_next_results()
#     while next_page_token:
#         # Must wait for the next_page_token to become valid
#         # See https://github.com/googlemaps/google-maps-services-python/issues/145
#         time.sleep(2)
#         next_page_token = _get_next_results(next_page_token=next_page_token)
    
#     df_businesses = pd.DataFrame(results)
#     df_businesses['searched_keyword'] = keyword
#     df_businesses['searched_zipcode'] = zipcode
#     return df_businesses

In [105]:
def search_place(keyword, zipcode):
    '''
    Inputs one keyword and one zip code
    Send a single request
    Outputs up to 20 results
    '''
    json_result = client.places(f'{keyword} near {zipcode}')
    df_businesses = pd.DataFrame(json_result['results'])
    
    # Add columns searched_keyword and searched_zipcode to the dataframe
    df_businesses['searched_keyword'] = keyword
    df_businesses['searched_zipcode'] = zipcode
    return df_businesses

In [107]:
def serach_all_places(list_of_keywords, list_of_zipcodes):
    '''
    Inputs a list of keywords (n) and a list of zip codes (m)
    Send (n*m) requests
    Outputs up to (n*m*20) results
    '''
    df = pd.DataFrame()
    for zipcode in list_of_zipcodes:
        for keyword in list_of_keywords:
            df = df.append(search_place(keyword, zipcode), ignore_index=True, sort=False) 
    return df

## Fetch data

In [142]:
#data = serach_all_places(LIST_OF_KEYWORDS, list_of_zcta)

In [144]:
# Check the shape of the data
data.shape

(9889, 16)

## Export data as .csv file

In [3]:
data.to_csv('../data/raw_google_data_nyc.csv')

NameError: name 'data' is not defined